In [ ]:
import awswrangler as wr
import pandas as pd
import datetime

In [ ]:
dfOriginal = wr.s3.read_parquet(path='s3://cchc-dw-qa-analytics/ts_real_presupuesto_gerencias/')

dfOriginal.columns.to_list()

In [ ]:
df = dfOriginal.copy()

df = df[(df.cr_nombre == 'CÁMARAS REGIONALES') & (df.fecha > '2021-01-01')]
#df = df[(df.area == 'CÁMARAS REGIONALES') & (df.fecha != 2023)]

df

In [ ]:
dfBNE = wr.s3.read_parquet(path='s3://cchc-dw-qa-analytics/bne_trabajadores/')

In [ ]:
df = dfBNE.copy()

df.columns.to_list()

8/2 
WORK

In [ ]:
dfBNE = wr.s3.read_parquet(path='s3://cchc-dw-dev-raw/bne/oficios_cvs_descargados/')


dfBNE

#antes del filtro sector 5821


In [ ]:
df = wr.s3.read_parquet(path='s3://cchc-dw-dev-raw/bne/bne_usuarios_empresas/')

df = df[df.is_activo == True]

df
#151 registros

In [ ]:
sii = wr.s3.read_parquet(path='s3://cchc-dw-qa-analytics/sii_socios_insolvencia/', columns=["rut","razon_social","tamano_de_empresa","rubro_economico","subrubro_economico","socio_cchc","estado"])
bolcon = wr.s3.read_parquet(path="s3://cchc-dw-dev-staging/boletin_concursal/insolvencias/")

In [ ]:
dfSii = sii.copy()
dfBolcon = bolcon.copy()

dfSii = dfSii[dfSii.estado == 'ACTIVO']
dfSii.fillna(value={'rut':-1})


dfSii = dfSii.drop_duplicates(subset=['rut'],keep='last')
df = dfBolcon.merge(dfSii,'left','rut')

#listadf=df.rut.drop_duplicates().to_list()
df = df[df.estado == 'ACTIVO']

df
#listaBolcon=dfBolcon.rut.drop_duplicates().to_list()

#df = df[df.rut.isnull()]
#for element in listadf:
#    if element not in listaBolcon:
#        print(element)

#print("bolcon: ",len(bolcon), " sii: ",len(sii), " df: " ,len(df))

In [ ]:
len(bolcon)


In [ ]:
nuevos_df = wr.s3.read_parquet(f"s3://cchc-dw-dev-raw/_bolcon/insolvencias/year=2022/month=11/day=14/", dataset=True)
nuevos_df["fecha_publicacion"] = pd.to_datetime(nuevos_df.fecha_publicacion, format="%d/%m/%Y")



anterior_df = wr.s3.read_parquet("s3://cchc-dw-dev-staging/boletin_concursal/insolvencias/",)

ruts_anterior = anterior_df[anterior_df.proceso_finalizado.isnull()].rut.fillna("CORRECCION:" + anterior_df.rol).drop_duplicates().to_list()
ruts_nuevos = nuevos_df.rut.fillna("CORRECCION:" + nuevos_df.rol).drop_duplicates().to_list()


#Buscar los ruts en el dataframe antiguos que no existan en el nuevo dataframe para marcarlos como finalizados, o sea, ya no son insolventes
ruts_finalizados = [x for x in ruts_anterior if x not in ruts_nuevos]

anterior_df.loc[ anterior_df.rut.isin(ruts_finalizados), "fecha_ejecucion"] = datetime.date(2022, 11, 7).strftime("%Y-%m-%d")
anterior_df.loc[ anterior_df.rut.isin(ruts_finalizados),  "proceso_finalizado"] = True

ruts_nuevos = [x for x in ruts_nuevos if x not in ruts_anterior]
nuevos_procesos = nuevos_df[nuevos_df.rut.fillna("CORRECCION:" + nuevos_df.rol).isin(ruts_nuevos)].copy()


df = pd.concat([anterior_df, nuevos_procesos])

print("Agregados", len(ruts_nuevos), "registros")
print(ruts_nuevos)
print("Marcados", len(ruts_finalizados), "como finalizados")

sii = wr.s3.read_parquet(path='s3://cchc-dw-qa-analytics/sii_socios_insolvencia/', columns=["rut","razon_social","tamano_de_empresa","rubro_economico","subrubro_economico","socio_cchc","estado"])

sii = sii[(sii.estado == 'ACTIVO') & (sii.rut.isin(ruts_nuevos))]
sii.fillna(value={'rut':-1})
sii = sii.drop_duplicates(subset=['rut'],keep='last')

df = df.merge(sii,'left','rut')

wr.s3.to_parquet(
    df=df,
    path="s3://cchc-dw-dev-staging/boletin_concursal/insolvencias/",
    dataset=True,
    mode="overwrite",
    table="bolcon_insolvencias_historico",
    database="staging_dev",
    schema_evolution=True,
    dtype={
        "fecha_ejecucion" : "string",
        "proceso_finalizado" : "boolean"
    }
)